In [1]:
print('runing')

runing


In [303]:
from partglot.utils.predict import get_loaded_model
import numpy as np
from src.helper.visualization import visualize_pointclouds_parts_partglot

import pymeshlab as pm
import torch
from partglot.utils.neural_utils import tokenizing
from sklearn.cluster import KMeans

def sort_arrays(arrays):
    ref_array = arrays[0]
    sorted_indices = ref_array.argsort()
    out = []
    for a in arrays:
        out.append(a[sorted_indices])
    return out

def random_sample_array(arr: np.array, size: int = 1, with_replacement:bool=True) -> np.array:
    if with_replacement:
        while len(arr) < size:
            arr = np.concatenate([arr, arr])
    return arr[np.random.choice(len(arr), size=size, replace=False)]

def cluster_supsegs(sorted_labels, sorted_pc, sup_seg_size=512):
    sup_segs, labels = [], []
    for lbl in np.unique(sorted_labels):
        indices = np.where(sorted_labels==lbl)[0]
        tmp_pc = sorted_pc[indices]
        tmp_lbl = sorted_labels[indices]
        sup_segs.append(random_sample_array(tmp_pc, sup_seg_size))
        # labels.append(random_sample_array(tmp_lbl, sup_seg_size))
        labels.append(lbl)
    return np.array(sup_segs), np.array(labels)


def get_attn_mask_objects(pc, pc2label):
    """
    Returns ordered point cloud and mask indices in our format.
    """
    stacked_pc = np.vstack(np.vstack(pc))
    
    arg_sort = pc2label.argsort()
    
    out_pc2label, out_pg_pc = pc2label[arg_sort], np.vstack(stacked_pc)[arg_sort]

    mask = {}
    for i, pn in enumerate(part_names):
        tmp = np.where(out_pc2label == i)[0]
        mask[pn] = [tmp.min(), tmp.max()]
    
    return {"mask_vertices": mask}, out_pg_pc



In [302]:
sample_idx = 0
run_gt = True

np.random.seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

part_names = ["back", "seat", "leg", "arm"]

part_semantic_groups = {
    "back": ["back"],
    "seat": ["seat"],
    "leg": ["leg", "wheel", "base"],
    "arm": ["arm"],
}

model_dir = "/home/bellatini/DL3D-Practical/models/pn_agnostic.ckpt"
# data_dir = "/home/bellatini/DL3D-Practical/data/partglot"
data_dir = "/home/bellatini/DL3D-Practical/data/partglot_100"

partglot, partglot_dm = get_loaded_model(data_dir=data_dir, model_path=model_dir)
sup_segs2label, pc2label = partglot.get_attn_maps()[sample_idx]
segs, masks = partglot_dm.h5_data['data'][sample_idx], partglot_dm.h5_data['mask'][sample_idx]

partglot.to(device)


batch_data = torch.from_numpy(partglot_dm.h5_data['data'][sample_idx:sample_idx+1]).unsqueeze(dim=1).float().to(device)
mask_data = torch.from_numpy(partglot_dm.h5_data['mask'][sample_idx:sample_idx+1]).unsqueeze(dim=1).float().to(device)

pc = np.vstack((np.vstack(np.vstack(batch_data.cpu().numpy()))))

kmeans = KMeans(n_clusters=25, random_state=1).fit(pc)
pc2sup_segs_kmeans = kmeans.labels_
np.unique(pc2sup_segs_kmeans, return_counts=True)

sorted_labels, sorted_pc = sort_arrays((kmeans.labels_, pc))

super_segs, pc2sup_segs = cluster_supsegs(sorted_labels, sorted_pc)

custom_ssegs_batch = torch.from_numpy(np.array([[super_segs]])).float().to(device)
custom_mask_batch = torch.from_numpy(np.array([[np.ones(custom_ssegs_batch.shape[2])]])).float().to(device)
custom_ssegs_batch.shape, custom_mask_batch.shape


if run_gt:
    final_ssegs_batch = batch_data
    final_mask_batch = mask_data 
    super_segs = batch_data[0][0].cpu().numpy() # gt_super_segs / super_segs
else:
    final_ssegs_batch = custom_ssegs_batch 
    final_mask_batch = custom_mask_batch 
    
attn_maps = []
for pn in part_names:
    text_embeddings = tokenizing(partglot_dm.word2int, f"chair with a {pn}").to(device)[None].expand(
        1, -1
    )
    tmp = partglot.forward(
        final_ssegs_batch, # custom_ssegs_batch / batch_data
        final_mask_batch, # custom_mask_batch / mask_data
        text_embeddings, True)
    attn_maps.append(tmp)
    

attn_maps_concat = torch.cat(attn_maps).max(0)[1].cpu().numpy()
sup_segs2label = np.squeeze(attn_maps_concat)
sup_segs2label

# CHANGE THIS TO TWEAT RUN!
K, n_points, coord = super_segs.shape

sup_segs2label_sorted=[] # pc2sup_segs: is actually pc2label
for lbl in sup_segs2label:
    tmp = np.ones(n_points) * lbl
    sup_segs2label_sorted.append(tmp)
    
sup_segs2label_sorted = np.concatenate(sup_segs2label_sorted).astype(int)

assign_ft = lambda x: sup_segs2label[x]

pc2label_sorted = assign_ft(pc2sup_segs.astype(int))

import numpy as np
pc2label_prefinal = []
for lbl in sup_segs2label:
    tmp = np.ones(512)*lbl
    pc2label_prefinal.append(tmp)
pc2label_prefinal = np.concatenate(pc2label_prefinal)

pc_final = np.vstack(super_segs)

# final_mask, final_pc = get_attn_mask_objects(super_segs, pc2label)
final_mask, final_pc = get_attn_mask_objects(pc_final, pc2label_prefinal) # pc2sup_segs: is actually pc2label

out = []
for s,f in final_mask['mask_vertices'].values():
    tmp = final_pc[s:f].astype(float)
    out.append(tmp)
    
visualize_pointclouds_parts_partglot(np.array(out))


/tmp/ipykernel_974786/3776868927.py:74: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  visualize_pointclouds_parts_partglot(np.array(out))


Output()

/tmp/ipykernel_974786/3618547245.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  visualize_pointclouds_parts_partglot(np.array(out))


Output()

/tmp/ipykernel_974786/3618547245.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  visualize_pointclouds_parts_partglot(np.array(out))


Output()

In [125]:
sup_segs2label_sorted[2000]

2

In [35]:
sup_segs2label_sorted.shape, np.vstack(super_segs).shape

((25600,), (25600, 3))

In [158]:
np.unique(sup_segs2label_sorted, return_counts=True), np.unique(pc2sup_segs_kmeans, return_counts=True), super_segs.shape


((array([0, 1, 2, 3]), array([6144, 6656, 6144, 6656])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
        dtype=int32),
  array([18022,   414,   152,   178,   132,   124,   202,   201,   142,
           105,   131,   125,   125,   105,    99,   100,   135,   138,
           111,   179,   124,   160,   113,   176,   163,   149,    96,
           191,    85,   131,   213,   145,   211,   170,    77,    82,
           235,   259,    91,   234,   398,   152,   139,   118,   126,
           127,    97,   139,   128,   151])),
 (50, 512, 3))

/tmp/ipykernel_974786/3618547245.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  visualize_pointclouds_parts_partglot(np.array(out))


Output()

/tmp/ipykernel_970082/1133928863.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(out)


array([array([[-0.07908174,  0.64580595,  0.36080492],
              [-0.16577069, -0.10363343,  0.12228566],
              [-0.18233299, -0.13928761,  0.15949082],
              ...,
              [ 0.13689284,  0.55354083,  0.32345203],
              [ 0.11216941,  0.64667374,  0.35172057],
              [ 0.13975477,  0.65062511,  0.35526568]]),
       array([[ 0.18890621, -0.39108256, -0.22447015],
              [ 0.18783529, -0.29047167, -0.214518  ],
              [ 0.18397629, -0.33517325, -0.25515747],
              ...,
              [ 0.14968847, -0.5149582 ,  0.18857178],
              [ 0.23388477, -0.49047479,  0.16833512],
              [ 0.26789564, -0.5133149 ,  0.0116857 ]]),
       array([[ 0.25526619, -0.82515508,  0.19333552],
              [ 0.22683148, -0.75497305,  0.19867164],
              [ 0.26904041, -0.80004388,  0.2088269 ],
              ...,
              [ 0.2166393 , -0.72088832, -0.29215953],
              [ 0.22620369, -0.72927099, -0.29806805],
    

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4,) + inhomogeneous part.

In [113]:
out.shape

(4,)

In [96]:
# out = []
# f = 0
# for i in range(50):
#     s, f = i*512, f + 512
#     out.append(y[s:f])
# out = np.array(out)

/tmp/ipykernel_970082/2765739158.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  visualize_pointclouds_parts_partglot(np.array(out)


Output()

array([0, 0, 0, ..., 0, 0, 0])

(25600,)

array([0, 1, 2, 3])

In [13]:
attn_maps_concat

array([[[0, 0, 2, 3, 1, 0, 2, 1, 3, 3, 2, 1, 2, 1, 3, 2, 0, 3, 0, 0, 2,
         2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]])

In [124]:
np.unique(final_pc, axis=0).shape

(1729, 3)

In [91]:
np.vstack(super_segs).shape

(12800, 3)

In [92]:
pc2label.shape[0] /2

6400.0

In [110]:
final_pc.shape

(12800, 3)

In [18]:
out = []
for s,f in final_mask['mask_vertices'].values():
    tmp = final_pc[s:f]
    out.append(tmp)

In [20]:
pc2label_ref = np.load("/home/bellatini/DL3D-Practical/Baselines/PartGlot/logs/pre_trained/pn_agnostic/12-12_14-37-13/pred_label/final/0_pc_label.npy")

In [22]:
pc2label_ref.shape

(2691,)

In [108]:
pc2label_ref.shape, np.unique(pc2label_ref)

((2691,), array([0, 1, 2, 3]))

In [95]:
# torch.cat(attn_maps).shape

In [19]:
visualize_pointclouds_parts_partglot(out)

Output()

In [36]:
visualize_pointclouds_parts_partglot(segs)

Output()

In [37]:
pc2label

array([3, 3, 3, ..., 0, 0, 0])